# Clustering Crypto

In [309]:
# Initial imports
import pandas as pd
import hvplot.pandas
import hvplot
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from unicodedata import name

### Deliverable 1: Preprocessing the Data for PCA

In [310]:
# Load the crypto_data.csv dataset.
file_path = "/Users/delonjackson/Desktop/GitHub/Cryptocurrencies/crypto_data.csv"
crypto_df = pd.read_csv(file_path)
crypto_df.head(10)

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015,2015 coin,X11,True,PoW/PoS,NaN,0
7,BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [311]:
crypto_df.drop(columns=['Unnamed: 0'], inplace=True)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [312]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df['IsTrading']==True]
crypto_df.head(10)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015 coin,X11,True,PoW/PoS,NaN,0
7,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [313]:
# Keep all the cryptocurrencies that have a working algorithm.
pd.isna(crypto_df['Algorithm'])


0       False
1       False
2       False
3       False
4       False
        ...  
1243    False
1244    False
1245    False
1246    False
1247    False
Name: Algorithm, Length: 1144, dtype: bool

In [314]:
# Remove the "IsTrading" column. 
crypto_df  = crypto_df.drop(['IsTrading'], axis=1) 
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0


In [315]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()
crypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
9,Litecoin,Scrypt,PoW,6.303924e+07,84000000
10,Dash,X11,PoW/PoS,9.031294e+06,22000000
11,Monero,CryptoNight-V7,PoW,1.720114e+07,0
12,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000


In [316]:
# Keep the rows where coins are mined.
crypto_df[crypto_df['TotalCoinsMined'] > 0]
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [317]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coins_names = crypto_df.filter(['CoinName'], axis = 1)
coins_names.head(10)

,CoinName
0,42 Coin
2,404Coin
4,808
5,EliteCoin
7,Bitcoin
8,Ethereum
9,Litecoin
10,Dash
11,Monero
12,Ethereum Classic


In [318]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop("CoinName", axis=1)
print(crypto_df.shape)
crypto_df.head(10)

(685, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
4,SHA-256,PoW/PoS,0.000000e+00,0
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0
9,Scrypt,PoW,6.303924e+07,84000000
10,X11,PoW/PoS,9.031294e+06,22000000
11,CryptoNight-V7,PoW,1.720114e+07,0
12,Ethash,PoW,1.133597e+08,210000000


In [319]:
# Use get_dummies() to create variables for text features.
crypto_scaled = pd.get_dummies(data=crypto_df, columns=['Algorithm', 'ProofType'])
print(crypto_scaled.shape)
crypto_scaled.head(10)

(685, 109)


,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
0,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0.000000e+00,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
11,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [320]:
# Standardize the data with StandardScaler().
crypto_scaled = StandardScaler().fit_transform(crypto_scaled)
print(crypto_scaled)

[[-0.10282804 -0.03823841 -0.03823596 ... -0.03823596 -0.03823596
  -0.03823596]
 [-0.07661326 -0.03823825 -0.03823596 ... -0.03823596 -0.03823596
  -0.03823596]
 [-0.10282804 -0.03823841 -0.03823596 ... -0.03823596 -0.03823596
  -0.03823596]
 ...
 [-0.0784756  -0.03823801 -0.03823596 ... -0.03823596 -0.03823596
  -0.03823596]
 [-0.10264677 -0.0382384  -0.03823596 ... -0.03823596 -0.03823596
  -0.03823596]
 [-0.10282485 -0.03823841 -0.03823596 ... -0.03823596 -0.03823596
  -0.03823596]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [321]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)

crypto_pca = pca.fit_transform(crypto_scaled)

crypto_pca

array([[-0.25460817,  1.21706093, -0.41385225],
       [-0.24032054,  1.21524311, -0.41431992],
       [-0.16226477,  0.83799313, -0.30237059],
       ...,
       [-0.09670735, -2.32610903,  0.26733469],
       [-0.25827722, -2.00449482,  0.25042975],
       [-0.12920049,  0.73468686, -0.1457278 ]])

In [322]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(data= crypto_pca, columns=['PC 1', 'PC 2', 'PC 3'], index=crypto_df.index)
pcs_df.head(10)

,PC 1,PC 2,PC 3
0,-0.254608,1.217061,-0.413852
2,-0.240321,1.215243,-0.414320
4,-0.162265,0.837993,-0.302371
5,0.275136,1.945880,-0.536135
7,-0.205165,-1.439999,0.105917
8,-0.257211,-2.098442,0.267158
9,-0.296898,-1.061009,-0.005585
10,-0.200874,1.429312,-0.421018
11,-0.253281,-2.200522,0.217714
12,-0.257135,-2.098451,0.267155


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [323]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range ok k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [324]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)
print(model.fit(pcs_df))

# Predict clusters
predictions = model.predict(pcs_df)
print(predictions)

KMeans(n_clusters=4, random_state=0)
[0 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 1 0 1 1 0 0 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1
 0 1 0 1 0 1 0 0 1 0 0 0 1 0 1 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 0 0 0 1 0
 1 0 1 0 0 1 1 1 1 0 0 1 1 0 1 0 0 0 1 1 1 1 0 0 0 0 0 1 1 1 0 0 1 1 1 0 0
 1 0 1 0 0 1 1 1 1 1 0 0 0 1 1 0 1 1 0 0 0 1 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1
 0 1 0 1 0 0 0 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 0 0 1 0 1 0 1 1 0 1
 0 0 0 1 1 0 1 0 0 1 0 0 0 1 1 0 1 1 1 0 0 1 1 0 0 1 0 0 0 0 1 1 0 1 1 1 1
 1 0 1 1 0 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0
 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 0 1 0 1 1 1 0 0 1 0 0 0 1 1 1 0 0 0 1 1 1 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 1 0 1
 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 1 0 0
 0 0 0 1 0 1 0 1 1 1 1 0 1 0 0 1 0 1 0 1 1 0 1 0 1 1 1 0 1 0 1 0 1 0 2 1 0
 1 1 1 1 1 1 0 0 1 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1
 1 1 0 0 0 0 0 1 0 1 1 1 0 0 1 1 1 0 0 1 0 0 1 1 0 1 0 1 1 0 0 

In [325]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
#coin_names = pd.DataFrame(crypto_df, index=crypto_df.index)

# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pcs_df ], axis=1, sort=False)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = coins_names ["CoinName"]

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["class"]=model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(685, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class
0,Scrypt,PoW/PoS,4.199995e+01,42,-0.254608,1.217061,-0.413852,42 Coin,0
2,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.240321,1.215243,-0.414320,404Coin,0
4,SHA-256,PoW/PoS,0.000000e+00,0,-0.162265,0.837993,-0.302371,808,0
5,X13,PoW/PoS,2.927942e+10,314159265359,0.275136,1.945880,-0.536135,EliteCoin,0
7,SHA-256,PoW,1.792718e+07,21000000,-0.205165,-1.439999,0.105917,Bitcoin,1
8,Ethash,PoW,1.076842e+08,0,-0.257211,-2.098442,0.267158,Ethereum,1
9,Scrypt,PoW,6.303924e+07,84000000,-0.296898,-1.061009,-0.005585,Litecoin,1
10,X11,PoW/PoS,9.031294e+06,22000000,-0.200874,1.429312,-0.421018,Dash,0
11,CryptoNight-V7,PoW,1.720114e+07,0,-0.253281,-2.200522,0.217714,Monero,1
12,Ethash,PoW,1.133597e+08,210000000,-0.257135,-2.098451,0.267155,Ethereum Classic,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [326]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="class",
    symbol="class",
    width=800,
    hover_name="CoinName", 
    hover_data=["Algorithm"],
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [327]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns= ['Algorithm', 'ProofType','TotalCoinsMined', 'TotalCoinSupply', 'PC 1', 'PC 2', 'PC 3','CoinName','class'], sortable=True, selectable=True)


:Table   [Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class]

In [328]:
# Print the total number of tradable cryptocurrencies.
print(f"There are {clustered_df.shape[0]} tradable cryptocurrencies")

There are 685 tradable cryptocurrencies


In [329]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaled_df = MinMaxScaler().fit_transform(clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])
print(scaled_df)

[[4.55364914e-16 5.94230127e-03]
 [5.76795558e-09 7.00182308e-03]
 [0.00000000e+00 5.94230122e-03]
 ...
 [1.51812440e-08 6.92655266e-03]
 [2.27682457e-10 5.94962775e-03]
 [1.08420218e-11 5.94243008e-03]]


In [330]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(data=scaled_df, columns= ['TotalCoinSupply', 'TotalCoinsMined'], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df["CoinName"] = clustered_df["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df["class"]= clustered_df['class']

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,class
0,4.553649e-16,0.005942,42 Coin,0
2,5.767956e-09,0.007002,404Coin,0
4,0.000000e+00,0.005942,808,0
5,3.406122e-06,0.035342,EliteCoin,0
7,2.276825e-10,0.005960,Bitcoin,1
8,0.000000e+00,0.006050,Ethereum,1
9,9.107298e-10,0.006006,Litecoin,1
10,2.385245e-10,0.005951,Dash,0
11,0.000000e+00,0.005960,Monero,1
12,2.276825e-09,0.006056,Ethereum Classic,1


In [332]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(
   x='TotalCoinsMined',
   y='TotalCoinSupply',
   hover_color=['CoinNamed'],
   by='class'
)

:NdOverlay   [class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply)